### 1. Install Torch and Opacus

In [ ]:
# !nvidia-smi

In [ ]:
# !python3 -V

In [ ]:
# !pip install --upgrade pip --user

In [ ]:
# !pip3 install torch torchvision --user
# !pip3 install torchsummary
# !pip3 install opacus --user

In [ ]:
# !python3.9 -m pip install -r requirements.txt --user

In [ ]:
# !pip install torch==1.13.0+cu117 torchvision==0.14.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html

### 3. Import the "src" Folder

In [ ]:
import sys
sys.path.append("src/")

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from ast import arg
from decimal import Decimal
import os
import copy
import time
import pickle
import numpy as np
import torch
from torch import nn
import math
import statistics

from torchsummary import summary

from options import args_parser
from update_s4 import LocalUpdate, ClientDataSet
from utils import test_inference
from models import CNNMnistRelu, CNNMnistTanh
from models import CNNFashion_MnistRelu, CNNFashion_MnistTanh
from models import CNNCifar10Relu, CNNCifar10Tanh
from utils import average_weights, exp_details
from datasets import get_dataset
from torchvision import models
from logging_results import logging

from opacus.dp_model_inspector import DPModelInspector
from opacus.utils import module_modification
from opacus import PrivacyEngine

In [ ]:
# torch.cuda.is_available()

In [ ]:
# print(torch.__version__)
# print(torch.version.cuda)
# print(torch.backends.cudnn.version())
# print(torch.cuda.get_device_name(0))
# print(torch.cuda.get_device_properties(0))

### 4. The Main Function to Run

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def main(args):
    
    ############# Common ###################
    print("Hello - World ")
    # args = args_parser()
    # print("Arguments: ", args)
    if args.gpu:
        torch.cuda.set_device(args.gpu)
    
    if torch.cuda.is_available():
      device = args.gpu
    else:
      device = 'cpu'   

    # device = torch.device(args.gpu if torch.cuda.is_available() else 'cpu')
    
    # load dataset and user groups
    train_dataset, test_dataset, user_groups, users_test_data = get_dataset(args)

    print("Length of user test dataset: ", users_test_data[0])

    # BUILD MODEL
    if args.model == 'cnn':
        # Convolutional neural netork
        if args.dataset == 'mnist':
            if args.activation == 'relu':
                global_model = CNNMnistRelu()
            elif args.activation == 'tanh':
                global_model = CNNMnistTanh()
            global_model.to(device)
            summary(global_model, input_size=(1, 28, 28), device=device)
        elif args.dataset == 'fmnist':
            if args.activation == 'relu':
                global_model = CNNFashion_MnistRelu()
            elif args.activation == 'tanh':
                global_model = CNNFashion_MnistTanh()
            global_model.to(device)
            summary(global_model, input_size=(1, 28, 28), device=device)
        elif args.dataset == 'cifar10':
            # global_model = models.resnet18(num_classes=10)  
            if args.activation == 'relu':
                global_model = CNNCifar10Relu()
            elif args.activation == 'tanh':
                global_model = CNNCifar10Tanh()
            global_model.to(device)
            summary(global_model, input_size=(3, 32, 32), device=device)
        elif args.dataset == 'dr':
            if args.model_type == 'Resnet50':
            # -----------------------ResNet50----------------------------
                global_model = models.resnet50(pretrained=True)
                set_parameter_requires_grad(global_model, True)
                num_ftrs = global_model.fc.in_features
                global_model.fc = nn.Linear(num_ftrs, 2)
            
            elif args.model_type == 'InceptionV3':
                #Be careful, expects (299,299) sized images and has auxiliary output
                global_model = models.inception_v3(pretrained=True)
                set_parameter_requires_grad(global_model, True)
                # Handle the auxilary net
                num_ftrs = global_model.AuxLogits.fc.in_features
                global_model.AuxLogits.fc = nn.Linear(num_ftrs, 2)
                # Handle the primary net
                num_ftrs = global_model.fc.in_features
                global_model.fc = nn.Linear(num_ftrs,2)
            
            elif args.model_type == 'Densenet121':
                global_model = models.densenet121(pretrained=True)
                set_parameter_requires_grad(global_model, True)
                num_ftrs = global_model.classifier.in_features
                global_model.classifier = nn.Linear(num_ftrs, 2)
                    
            elif args.model_type == 'Vgg16':
                global_model = models.vgg16(pretrained=True)
                set_parameter_requires_grad(global_model, True)
                num_ftrs = global_model.classifier[6].in_features
                global_model.classifier[6] = nn.Linear(num_ftrs,2)
            
            elif args.model_type == 'Alexnet':
                global_model = models.alexnet(pretrained=True)
                set_parameter_requires_grad(global_model, True)
                num_ftrs = global_model.classifier[6].in_features
                global_model.classifier[6] = nn.Linear(num_ftrs,2)
            
            elif args.model_type == 'Squeezenet1.1':
                global_model = models.squeezenet1_1(pretrained=True)
                set_parameter_requires_grad(global_model, True)
                global_model.classifier[1] = nn.Conv2d(512, 2, kernel_size=(1,1), stride=(1,1))
                global_model.num_classes = 2
            elif args.model_type == 'Googlenet':
                global_model = models.googlenet(pretrained=True)
                set_parameter_requires_grad(global_model, True)
                # Handle the auxilary net
                # num_ftrs = global_model.AuxLogits.fc.in_features
                # global_model.AuxLogits.fc = nn.Linear(num_ftrs, 5)
                # Handle the primary net
                num_ftrs = global_model.fc.in_features
                global_model.fc = nn.Linear(num_ftrs,2)
                
            global_model.to(device)
            # summary(global_model, input_size=(3, 224, 224), device=device)
    else:
        exit('Error: unrecognized model')
    ############# Common ###################

    ######### DP Model Compatibility #######
    if args.withDP:
        try:
            inspector = DPModelInspector()
            inspector.validate(global_model)
            print(args.model_type + ": Model's already Valid!\n")
        except:
            global_model = module_modification.convert_batchnorm_modules(global_model)
            inspector = DPModelInspector()
            print(f"Is the model valid? {inspector.validate(global_model)}")
            print("Model is convereted to be Valid!\n")        
    ######### DP Model Compatibility #######

    

    ######### Local Models and Optimizers #############
    local_models = []
    local_optimizers = []
    local_privacy_engine = []

    for u in range(args.num_users):
        local_models.append(copy.deepcopy(global_model))

        if args.optimizer == 'sgd':
            optimizer = torch.optim.SGD(local_models[u].parameters(), lr=args.lr, 
                                        momentum=args.momentum)        
        elif args.optimizer == 'adam':
            optimizer = torch.optim.Adam(local_models[u].parameters(), lr=args.lr)             

        if args.withDP:
          # This part is buggy intentionally. It makes privacy engine avoid giving error with vhp.
            
            privacy_engine = PrivacyEngine(
                local_models[u],
                batch_size = int(len(user_groups[u])*args.sampling_prob), 
                sample_size = len(user_groups[u]), 
                alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)),
                noise_multiplier = args.noise_multiplier/np.sqrt(args.num_users),
                max_grad_norm =  args.max_grad_norm,
            )

            privacy_engine.attach(optimizer)            
            local_privacy_engine.append(privacy_engine)

        local_optimizers.append(optimizer)


    if args.optimizer == 'sgd':
        g_optimizer = torch.optim.SGD(global_model.parameters(), lr=args.lr, 
                                    momentum=args.momentum)        
    elif args.optimizer == 'adam':
        g_optimizer = torch.optim.Adam(global_model.parameters(), lr=args.lr)        
    if args.withDP:
        local_dataset_size = int(len(train_dataset)/args.num_users)
        actual_train_ds_size = local_dataset_size*args.num_users
        global_privacy_engine = PrivacyEngine(
            global_model,
            batch_size = int(actual_train_ds_size*args.sampling_prob),
            sample_size = actual_train_ds_size,
            alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)),
            noise_multiplier = args.noise_multiplier,
            max_grad_norm =  args.max_grad_norm)  
        global_privacy_engine.attach(g_optimizer)
    ######## Local  Models and Optimizers #############

    # Training
    train_loss = []
    test_log = []
    epsilon_log = []
    
    # print("Avg batch_size: ", int(actual_train_ds_size*args.sampling_prob))
    
    theta = 3.001 # Theta
    phi = 0.01 # Phi
    post_median_acc = 0.0

    # eligible_users = np.random.choice(range(args.num_users),
    #                                   max(int(args.frac * args.num_users), 1),
    #                                   replace=False)

    for round in range(args.rounds):
        print("--->Round: ", round)    
        ## Sample the users ##        
        eligible_users = np.random.choice(range(args.num_users),
                                      max(int(args.frac * args.num_users), 1),
                                      replace=False)
        #####
        local_weights, local_losses = [], []
        A_mpost = [] # median post-trained accuracy
        print("---->Number of current Users: ", len(eligible_users))

        for u in eligible_users:
            # #---------------------Checkpoint-1----------------------------
            print("Checkpoint-1: ")
            # client_dataset = ClientDataSet(args = args, dataset = train_dataset, idxs = user_groups[u])
            A_upre, _loss = test_inference(args, global_model, users_test_data[u])

            print("A_upost: {:.2f},  post_median_acc: {:.2f}, Theta: {:.2f}".format(A_upre, post_median_acc, theta))
            # print("Round: ", round)
            print("A_mpost: ", A_mpost)
            diff1 = post_median_acc - theta
            print("Diff1: ", diff1)

            if (A_upre > diff1):
                print("@CHECKPOINT 1 is satisfied.")
            # ------------------------------------------------------------
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

                local_model = LocalUpdate(args=args, dataset=train_dataset, 
                                            u_id=u, idxs=user_groups[u], 
                                            sampling_prob=args.sampling_prob,
                                            optimizer = local_optimizers[u])

                w, loss, local_optimizers[u] = local_model.update_weights(
                                                        model=local_models[u],
                                                        global_round=round)
                
                # l_acc, l_loss = test_inference(args, local_models[u], test_idxs)  
                
                # local_weights.append(copy.deepcopy(w))
                # local_losses.append(copy.deepcopy(loss))

                # print("Accuracy: ", l_acc, " Loss: ", l_loss)
                    #---------------------Checkpoint-2----------------------------
                u_model = local_models[u]
                u_model.load_state_dict(copy.deepcopy(w))
                A_upost, _loss = test_inference(args, u_model, users_test_data[u])
                diff2 = A_upost - A_upre
                print("Checkpoint-2: ")
                print("User: ", u, " Loss: ", _loss)
                print("A_upost: {:.2f},  A_upre: {:.2f}".format(A_upost, A_upre))
                print("Round: {:.2f}, Diff2: {:.2f},  Phi: {:.2f}".format(round, diff2, phi))

                if math.fabs(diff2) > phi:
                    print("@CHECKPOINT 2 is satisfied.")
                    A_mpost.append(A_upost)
                    local_weights.append(copy.deepcopy(w))
                    local_losses.append(copy.deepcopy(loss))
                else:
                    eligible_users = eligible_users - [u]
                # ------------------------------------------------------------
            else:
                eligible_users = eligible_users - [u] # If not allowed for next global round
            if len(A_mpost) and len(eligible_users) > 0:
                post_median_acc = statistics.median(A_mpost) # Get Median accuracy of all clients updated model
        
        if len(local_weights) > 0:
            # update global weights by averaging the local weigths
            global_weights = average_weights(local_weights)

            # update global model
            global_model.load_state_dict(global_weights)
            for u in range(args.num_users):
                local_models[u].load_state_dict(global_weights)

            # if round != 0:
            torch.cuda.empty_cache()          
            loss_avg = sum(local_losses) / len(local_losses)        
            train_loss.append(loss_avg)

            _acc, _loss = test_inference(args, global_model, test_dataset)
            print("Global: ")  
            print("Accuracy: ", _acc, "Loss: ", _loss)    

            test_log.append([_acc, _loss])  
        
            if args.withDP:
                global_privacy_engine.steps = round+1
                epsilons, _ = global_privacy_engine.get_privacy_spent(args.delta)                                        
                epsilon_log.append([epsilons])
            else:
                epsilon_log = None

            logging(args, round, train_loss, test_log, epsilon_log)
            print(global_privacy_engine.steps)

### 5. The Chosen Settings for the Experimt

In [ ]:
import argparse

parser = argparse.ArgumentParser()

## etc.
parser.add_argument('--virtual_batch_size', type=int, default=1, 
                    help='DP VIRTUAL_BATCH_SIZE')

## Optimizer
parser.add_argument('--optimizer', type=str, default='sgd', help="type \
                    of optimizer")

parser.add_argument('--lr', type=float, default=.002,
                    help='learning rate')

parser.add_argument('--momentum', type=float, default=0.9,
                    help='SGD momentum (default: 0.0)')


# model arguments
parser.add_argument('--model', type=str, default='cnn', help='model name')

parser.add_argument('--activation', type=str, default="relu",
                    help='SGD momentum (default: 0.0)')

# other arguments
parser.add_argument('--dataset', type=str, default='dr', help="name \
                    of dataset")

parser.add_argument('--unequal', type=int, default=0,
                    help='whether to use unequal data splits for  \
                    non-i.i.d setting (use 0 for equal splits)')
                                  
## DP arguments
parser.add_argument('--withDP', type=int, default=1, help='WithDP')
parser.add_argument('--max_grad_norm', type=float, default= 1.2, help='DP MAX_GRAD_NORM')

parser.add_argument('--sampling_prob', type=int, default=0.01 , help='sampling_prob')
parser.add_argument('--frac', type=float, default=1.0, help='the fraction of clients: C')

# Can be changed the following values for model training
parser.add_argument('--parent_dir', type=str, default='/home/siu856533724/code', help='Parent directory')
parser.add_argument('--data_set', type=int, default=1, help="Select Dataset which will be trained.")
parser.add_argument('--data_dir_1', type=str, default="/home/siu856533724/dataset/resized/original/resized_train/", help="Directory where whole dataset is stored.")
parser.add_argument('--data_dir_2', type=str, default="/home/siu856533724/dataset/aptos2019/images/", help="Directory where whole dataset is stored.")

parser.add_argument('--train_csv', type=str, default="/train_labels_2.csv", help="The Label file for training dataset.")
parser.add_argument('--test_csv', type=str, default="/train_labels_2.csv", help="The Label file for test dataset.")
# federated arguments (Notation for the arguments followed from paper)
parser.add_argument('--local_bs', type=int, default=1, help="local batch size: B")
parser.add_argument('--local_test_split', type=float, default=0., help='DP DELTA')  
parser.add_argument('--iid', type=int, default=1, help='Default set to IID. Set to 0 for non-IID.')
parser.add_argument('--num_shards', type=int, default=100, help='Its needed for non-IID.')
                    
parser.add_argument('--sub_dataset_size', type=int, default=-1, help='[6250, 10000, 12500, 18750, 31250]')
parser.add_argument('--rounds', type=int, default=10, help="number of rounds of training")
parser.add_argument('--num_users', type=int, default=100, help="number of users: K")
parser.add_argument('--local_ep', type=int, default=10, help="the number of local epochs: E")
parser.add_argument('--noise_multiplier', type=float, default=0.5, help='DP NOISE_MULTIPLIER')
parser.add_argument('--delta', type=float, default=0.00001 , help='DP DELTA')
parser.add_argument('--model_type', type=str, default='Squeezenet1.1', 
                        help='Image classification model name[Resnet50, Alexnet, Vgg16, Squeezenet1.1, Densenet121, #InceptionV3, #googelnet]')
parser.add_argument('--gpu', default="cuda:1", help="To use cuda, set \
                    to a specific GPU ID. Default set to use CPU.")

parser.add_argument('--dr_from_np', type=float, default=1, help='for diabetic_retinopathy dataset')
parser.add_argument('--train_images_npy', type=str, default='/data/train/dr_train_images_1_class2_', help='Converted data set')
parser.add_argument('--train_labels_npy', type=str, default='/data/train/dr_train_labels_1_class2_', help='Converted data label')

parser.add_argument('--test_images_npy', type=str, default='/data/test/dr_test_images_1_class2_', help='Converted data set')  
parser.add_argument('--test_labels_npy', type=str, default='/data/test/dr_test_labels_1_class2_', help='Converted data label')   

parser.add_argument('--exp_name', type=str, default="DPSGD", help="The name of current experiment for logging.")
parser.add_argument('--class_num', type=int, default=2, help='how many classes have to be predicted.')
args = parser.parse_args([])
main(args)